In [70]:
import spacy
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import re

# Load only once
try:
    print(nlp)
except:
    nlp = spacy.load("en_core_web_md")



# Only the visible text is to be retrieved
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    title = soup.title.string
    return u" ".join(t.strip() for t in visible_texts), title

def tokenize(text):
    t0 = []
    # Split the text into words separated by any of these chars
    p = re.compile('[-_\", {};:=?\[\]\(\)\'.]')
    
    # Get the word list into an array, t
    t = p.split(text)
    lt = len(t)

    # Add the 4+ chars words into an array, t1. Discard the smaller words
    for i in range(lt):
        w = t[i]
        lw = len(w)
        if(lw > 3):
            t0.append(w)
    return t0

def Lemmatise(text):
# Implementing lemmatization
    lem = nlp(text)
    # finding lemma for each word
    for word in lem:
        print(word.text,word.lemma_)
    pass
def ReadUrls(i, j):
    url1 = urls[i]
    url2 = urls[j]
    global html1, html2, t1, t2
    t1 = []
    t2 = []
    response =  urllib.request.urlopen(url1)
    html1 = response.read()
    response =  urllib.request.urlopen(url2)
    html2 = response.read()
    text1, title1 = text_from_html(html1)
    text1 = Lemmatise(text1)
    text2, title2 = text_from_html(html2)
    t1 = tokenize(text1)
    t2 = tokenize(text2)  
    print(url1, ' ', url2)        

def GetDocSegments():
    global doc1_segs, doc2_segs, l, m
    # Split and save the text as 100 word segments
    doc1_segs = []
    doc2_segs = []
    text = ''
    # Split t1 and t2 into text of 100 words
    l = len(t1)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t1[j] + ' '
        doc1_segs.append(text)
        text = ''

    l = len(t2)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t2[j] + ' '
        doc2_segs.append(text)
        text = ''

    l = len(doc1_segs)    
    m = len(doc2_segs) 
    print(l,m)

def CalculateSimilarity():
    sim = 0.00
    sims = []
    tot = 0.00
    k = 10
    if (l < k):
        k = l
    if (m < k):
        k = m
    for i in range(k):
        for j in range(k):
            doc1 = nlp(doc1_segs[i])
            doc2 = nlp(doc2_segs[j])
            sim = doc1.similarity(doc2)
            sims.append(sim)
    sims.sort(reverse=True)
    for i in range(k):
        tot += sims[i]
    avg = tot/k
    print(avg)

# Globals
urls = [
"https://en.wikipedia.org/wiki/Machine_learning", # 0
"https://www.sas.com/en_au/insights/analytics/machine-learning.html", # 1
"https://en.wikipedia.org/wiki/SpaceX", # 2
"https://en.wikipedia.org/wiki/London", # 3
"https://en.wikipedia.org/wiki/England",# 4
"https://www.webgenie.com/details.html", # 5
"https://en.wikipedia.org/wiki/Titanic_(1997_film)", # 6
"https://en.wikipedia.org/wiki/Titanic_(1953_film)", # 7
"https://simple.wikipedia.org/wiki/Titanic_(1997_movie)", # 8
"https://en.wikipedia.org/wiki/Star_Wars", # 9
"https://en.wikipedia.org/wiki/List_of_Star_Wars_films" # 10
]


ReadUrls(9, 9)
GetDocSegments()
CalculateSimilarity()

https://en.wikipedia.org/wiki/Star_Wars   https://en.wikipedia.org/wiki/Star_Wars
127 127
1.0
